# NERC Field Spectroscopy Introduction to Field Spectroscopy Practical -- Wavelength 2023

## Aim of today's session
This notebook will take you through the basic steps of visualising data from a field spectrometer, and will cover some of the steps that you can take to produce data products such as spectral indices and convolution to satellite sensor measurements. 
The data used in this practical was gathered using an SVC HR-1024i spectrometer with leaf clip attachment -- the same setup that you will be using in the in person practical -- measuring the spectral response of various leaf samples gathered in September 2022.

## Using as part of the tutorial sets

This workbook is designed such that you can use the functions and methodology set out here as part of the questions that will be asked throughout the workbook. 

You can also paste your own data acquired in your work into the "data" folder and run through the processing with minimal modification. If you would like assistance in doing this in the future please email us at fsf@nerc.ac.uk

## First Steps -- Importing Modules

The first stage in any workflow in Python is to import the modules (which are collections of functions) that you will be using. In this analysis, we import a number of modules, the function of which is described in the comments (the light cyan text prefixed by the hash symbol). Note, modules must be installed first before use -- this can be handled by the conda package manager, or the pip package manager (please get in touch with us if you require help in setting this up). Here, the most important module to note is SpecDAL, a module designed to handle and process field spectroscopy data. To run the cell, press Shift and ENTER, or press Run in the ribbon above.

In [ ]:
import os    #a module that allows us to "talk" with our operating system, neccessary for handling paths, files and so on
from pathlib import Path #a more specific path handling module that allows us to quickly create path names
from specdal import Collection, Spectrum, read #We import our main package, specdal, and ask to only import certain key functions
import pandas as pd   #Pandas is a powerful module for the handling of data sets
from matplotlib import pyplot as plt   #matplotlib emulates the functions of MATLAB. Here, we tell matplotlib to only import one of it's functions, the pyplot function, and to import it with the identifier 'plt'
from matplotlib.pyplot import ylabel, xlabel, title, legend   #We go even further by asking pyplot to only import some of it's subfunctions
from scipy.signal import savgol_filter   #We will use this to smooth our data
import numpy as np #We require numpy for integration functions that will be used during convolution

## SpecDAL

We will now use the SpecDAL package to read our .sig files, and assign them to a collection of spectra which we can analyse and process further. Our .sig files are located in the Data directory.

In [ ]:
Leaves = Collection(name='Leaves', directory ='Data')

Let's now take a look at this spectra collection. First of all, let's have a look at the header information for the first 3 spectra in our collection. This not only prints the data itself (in the "measurements" category), but also the metadata associated with the file, such as the measurement type (here, pct_reflect refers to this being reflectance measurements), and such things as GPS information, which can be useful when matching the GPS co-ordinates with imagery.

In [ ]:
print(type(Leaves.spectra))
for s in Leaves.spectra[0:3]:
    print(s)

We can take a visual look at the data too...

In [ ]:
Leaves.plot(figsize=(15, 6), legend = True, ylim=(0,1), xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Relative Reflectance")
plt.legend(bbox_to_anchor=(1.05, 1), loc = "upper left")
plt.show()

## Grouping Data

It can be difficult to assess your data by viewing all spectra at once. We will group the data based on the vegetation type using SpecDAL's groupby function. This groups data based on their file names, in this case the name before the "_" separator e.g. "redleaf".


In [ ]:
groups = Leaves.groupby(separator='_', indices=[0])
group_names = list(groups.keys())
print(group_names)

We can now limit our graph to show only one end member type, e.g. 'readleaf'.

In [ ]:
groups['redleaf'].plot(figsize=(15, 6), legend = True, ylim=(0,1), xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Relative Reflectance")
plt.legend(bbox_to_anchor=(1.05, 1), loc = "upper left")
plt.show()

**Question** -- Modify the above code so that the graph displays the spectra for the 'greenleaf' group.

In [ ]:
#Use this empty cell to generate your code for the answer above. The questions below also contain empty cells immediately after for you to code in






We can then average each of these groups to produce a collection of means, called "means". 

In [ ]:
means = Collection(name='means')
for group_key, group_collection in groups.items():
    means.append(group_collection.mean())

means.plot(title='Group means', figsize=(15, 6), ylim=(0, 1),xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Relative Spectral Reflectance")
plt.show()

## Interpolation
Look again at the print out for your data spectra, using data.head(10) function to print off the first 10 rows of data.

In [ ]:
means.data.head(10)

For SVC instruments, the steps between the wavelengths correspond to the spectral resolution of the instrument, and are not resolved to 1 nm spacing. We want to interpolate reflectance measurements that correspond to wavelengths with 1.0 nm spacing, a process called interpolation. This can be done using specdal as follows with the interpolate function (ignore the warning messages!).

In [ ]:
means.interpolate(spacing=1, method='linear')
means.data.head(10)

## Relative vs Absolute Reflectance

Notice from our previous graphs that the y-axes are labelled "Relative Reflectance". This is because these spectra were recorded relative to the reflectance of the white Spectralon panel. We take measurements relative to this panel to approximate the total irradiance coming from the sun and hitting the object you are interested in taking a spectral measurement of. Because the panel does not reflect 100% of the light that hits it in a completely uniform manner we need to adjust our "Relative Reflectance" measurements using the panel's known, laboratory calibrated reflectance to convert our field measurements to absolute reflectance.

We can derive the absolute reflectance then by multiplying our field measurements by the spectral reflectance of the panel for each wavelength. We can use this file to convert our measurements to absolute reflectance, pulling data from our calibration certificate, SRT_44.csv. Note -- we are creating new dataframes that will host the corrected values, but the original dataframe can be corrected without having to assign a new identifier!

In [ ]:
reference_panel = pd.read_csv("SRT_44.csv", index_col = "wavelength")
Absolute_Leaf_Means = means.data.mul(reference_panel['Reflectance'], axis = 0)

## Other prcoessing steps to consider
Depending on the spectrometer that you use, you may have to conduct 'overlap matching' and 'jump correction'. A typical field spectrometer covering the 350 nm to 2500 nm range consists of three spectrometers -- one covering the VNIR range (350 - 1000 nm), and two for the SWIR range (usually classed as SWIR-A, covering 1000 nm to 1800 nm, and SWIR-B, covering the 1800 nm to 2500 nm range. These spectrometers overlap in their ranges, leading to the zig-zags which you can see when plotting non-overlap corrected data. We can correct this using specdal's overlap stitching function, stitch, and then resolve 'jumps' in the subsequent data using the jump correction function, jump_correct.

For field measurements not using a leaf clip probe, where absorption of solar irradiance by water vapour in the atmopshere occurs, noisy regions ~1400nm and ~1900nm can occur in your data. Including these regions in your data can interfere with classification or use in indices. Sometimes, it's appropriate to remove these regions. For this tutorial, we won't do so, but if you need any help on this issue, please contact FSF at fsf@ed.ac.uk.

## Final processing stages
Let's take a look at our final data output, where raw .sig files have been amalgamated into groups, averaged, interpolated, and converted to absolute reflectance...

In [ ]:
Absolute_Leaf_Means.plot(title='Leaf Means', figsize=(15, 6), ylim=(0, 1),xlim=(350, 2500))
xlabel("Wavelength (nm)")
ylabel("Absolute Spectral Reflectance")
plt.show()

We can now export this collection as a .csv file for processing outwith Python --

In [ ]:
Absolute_Leaf_Means.to_csv("Leaf_Means.csv")

## Vegetation Indices
With your data now processed and converted into absolute spectral reflectance, we can look more closely at differences between vegetation types. We can use dimensionality reduction methods such as spectral indices to do this --

https://fsf.nerc.ac.uk/resources/learning/HSI.shtml

A number of spectral indices have been designed to highlight different vegetation properties. In this next section, we will use some of them to explore differences between our vegetation types. For a full description of the indices used, please visit: https://www.l3harrisgeospatial.com/docs/NarrowbandGreenness.html

One of the most commonly used vegetation indices is the Normalized Difference Vegetation Index (NDVI). When using Sentinel-2 data, the equation to calculate NDVI is -- 

\begin{align}
NDVI = \dfrac{B_8 - B_4}{B_8 + B_4}
\end{align}

where Band 8 is equivalent to 842 nm, and Band 4 is equivalent to 665 nm. 

Using the inbuilt Python function .iloc, we can calculate NDVI for our leaf means. First, let's look at what iloc does.

In [ ]:
Absolute_Leaf_Means.iloc[502]

Here, iloc takes the index number for the Absolute_Leaf_Means dataframe, and reports what is at that value -- here, the reflectance of our different leaf groups, and also, as 'Name', the wavelength at which those reflectances were measured. You should see that 'Name' equals 842.0 nm. Notice the offset between the iloc value given and our wavelength value -- 340, which is the wavelength value at which our dataframe begins.

With this in mind, we can now proceed to calculate NDVI:

In [ ]:
Absolute_Leaf_Means.loc['NDVI'] = ((Absolute_Leaf_Means.iloc[502] - Absolute_Leaf_Means.iloc[325]) / 
                                  (Absolute_Leaf_Means.iloc[502] + Absolute_Leaf_Means.iloc[325])) 

print(Absolute_Leaf_Means.loc['NDVI'])

**Question** -- The Carotenoid Reflectance Index looks at the the concentrations of carotenoids in vegetation. Carotenoids function in light absorption processes in plants, as well as in protecting plants from the harmful effects of too much light. Higher CRI1 values mean greater carotenoid concentration relative to chlorophyll. The value of this index ranges from 0 to more than 15. The common range for green vegetation is 1 to 12:

\begin{align}
CRI = \dfrac{1}{510} - \dfrac{1}{550}
\end{align}

Determine the Carotenoid Reflectance Index for your leaf data set.

**Question** -- The Plant Senescence Reflectance Index maximizes the sensitivity of the index to the ratio of bulk carotenoids (for example, alpha-carotene and beta-carotene) to chlorophyll. An increase in PSRI indicates increased canopy stress (carotenoid pigment), the onset of canopy senescence, and plant fruit ripening. Applications include vegetation health monitoring, plant physiological stress detection, and crop production and yield analysis. The value of this index ranges from -1 to 1. The common range for green vegetation is -0.1 to 0.2:

\begin{align}
PSRI = \dfrac{680 - 500}{750}
\end{align}

Determine the Plant Senescence Reflectance Index for your leaf data set.


## Convolution
If your research relates to specific multispectral imaging sensors e.g. Sentinel 2, it can be useful to resample your hyperspectral data so that it matches the bands of your specific sensor. We can do this by convolving the hyperspectral data to a multispectral sensor's "spectral response function".

The Field Spectroscopy Facility has a Python package available for swift convolution of hyperspectral data to Sentinel-2 and WorldView multispectral imagery. For this tutorial, we'll look at how this package works, by convolving the data for one band of Sentinel-2 -- Band 4.

Band 4 of the Sentinel-2 sensor is also know as the "Red" band. The centre wavlength -- the wavelength at which it has the most sensitivty -- is 665 nm.

The band, however, is sensitive to other wavelengths (what is termed as its 'spectral response feature'), from 636 nm to 684 nm. The spectral response feature for Band 4, and other Sentinel 2 bands, is shown below.

![image](Image/Sentinel_2_Bands.png)

In order to accurately compare our hyperspectral data to Band 4, we must convolve the hyperspectral data within the wavelength regions of Band 4's SRF. This requires knowledge of the varying sensitvity of the band. The .csv file 'Sentinel 2 SRF.csv' contains this information. 


In [ ]:
Sentinel_2_bands = pd.read_csv('Sentinel 2 SRF.csv', index_col=0)

Knowing the starting and ending wavelength of the SRF of interest, we can now conduct convolution, which -- mathematically -- is the result of the trapezodial integration of the hyperspectral data for the SRF region divided by the result of the trapezoidial integration of the SRF itself.

In [ ]:
Absolute_Leaf_Means.loc['Band 4 - Red'] = (np.trapz((Absolute_Leaf_Means.iloc[306:344]), axis = 0)) / (np.trapz((Sentinel_2_bands.iloc[306:344, 3]), axis = 0))
print(Absolute_Leaf_Means.loc['Band 4 - Red'])

Considering this code in more detail -- **np.trapz** is the mathematical function conducted on the data; **.iloc[296:344]** gives the region to conduct the convolution (iloc[306] is equal to wavelength 636 nm, and iloc[344] is equal to wavelength 684 nm); and the **,3** after **296:344** for the integration of the Sentinel 2 bands relates to the column number in which the SRF for Band 4 is stored (the relation between column number and band in the SRF file is **Band Number - 1**). The final output gives the reflectance value of the convoluted hyperspectral data. 

**Question** -- We will now repeat the process as outlined above, but for Band 3, 'Green'. Band 3 has a centre wavlength at 560 nm. The SRF for Band 3 extends from 538 nm to 583 nm. Modifying the code above, create a row in Absolute_Leaf_Means called 'Band 3 - Green' that provides the output of the convolution of the hyperspectral data to the Band 3 SRF. 

**Question** -- Repeat the same process for Sentinel-2 Band 2, 'Blue'. The centre wavelength for Band 2 is 520 nm, and the SRF extends from 439 nm to 533 nm. 

**Bonus Question** -- With the Red, Green and Blue bands calculated, you can deduce the average colour of each leaf type by considering that in the RGB colour model, values are scaled from 0 to 255. 

## Conclusion

We have now gone from raw field spectroscopy data to post-processed spectra, that have then formed the basis of further investigation, such as NDVI and convolution. With these tools, you can start using your spectral data to answer many questions in different fields where spectroscopy is utilized, such as geology, ecology, or marine science!

As always, you can contact the Field Spectroscopy Facility for any question regarding both acquistion and processing of spectral data, at fsf@ed.ac.uk.